# Game Master Agents Exercise

In this exercise, we'll create game master agents that:
1. Run tabletop RPG scenarios
2. Use plugins for game mechanics
3. Track game state and player actions

Focus on combining agents with plugins for dynamic gameplay!

In [ ]:
import os
import sys
import random
from typing import Annotated

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)

sys.path.append(grandparent_dir)

### 1. Setup  

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

kernel = Kernel()
kernel.add_service(AzureChatCompletion(service_id="agent"))

# Helper function for chat interaction
async def chat_with_agent(agent: ChatCompletionAgent, message: str, chat: ChatHistory):
    """Function to handle agent interaction"""
    chat.add_user_message(message)
    print(f"Player: {message}")
    
    chunks = []
    async for chunk in agent.invoke_stream(chat):
        chunks.append(chunk)
        print(chunk.content, end="", flush=True)
    print("\n")
    
    complete_response = "".join([chunk.content for chunk in chunks])
    chat.add_assistant_message(complete_response)

### 2. Create Game Plugins (Experiment with these!)

Try modifying these plugins to add new game mechanics!

In [ ]:
class DicePlugin:
    """Plugin for dice rolling and random events"""
    
    @kernel_function(description="Roll dice with specified number of sides")
    def roll_dice(
        self,
        sides: Annotated[int, "Number of sides on the dice"],
        count: Annotated[int, "Number of dice to roll"] = 1
    ) -> str:
        """Roll specified dice and return results."""
        rolls = [random.randint(1, sides) for _ in range(count)]
        total = sum(rolls)
        return f"Rolled {count}d{sides}: {rolls} = {total}"

    @kernel_function(description="Generate a random event based on location")
    def random_event(
        self,
        location: Annotated[str, "Current game location"]
    ) -> str:
        """Generate a random event appropriate for the location."""
        events = {
            "forest": ["Animal encounter", "Strange sounds", "Hidden path", "Weather change"],
            "dungeon": ["Trap activated", "Monster appears", "Hidden treasure", "Cave-in"],
            "city": ["Street festival", "Pickpocket attempt", "Guard patrol", "Merchant sale"],
            "default": ["Mysterious noise", "Strange sight", "Unexpected discovery", "Sudden change"]
        }
        location_events = events.get(location.lower(), events["default"])
        return f"Event: {random.choice(location_events)}"

class CombatPlugin:
    """Plugin for handling combat mechanics"""
    
    @kernel_function(description="Calculate attack result")
    def attack_roll(
        self,
        attacker: Annotated[str, "Name of attacker"],
        target: Annotated[str, "Name of target"],
        bonus: Annotated[int, "Attack bonus"] = 0
    ) -> str:
        """Calculate and return attack result."""
        roll = random.randint(1, 20)
        total = roll + bonus
        if roll == 20:
            return f"Critical Hit! {attacker} rolled {roll} + {bonus} = {total} against {target}"
        elif roll == 1:
            return f"Critical Miss! {attacker} rolled {roll} + {bonus} = {total} against {target}"
        else:
            return f"{attacker} rolled {roll} + {bonus} = {total} against {target}"

    @kernel_function(description="Calculate damage result")
    def damage_roll(
        self,
        damage_dice: Annotated[str, "Dice notation (e.g., '2d6')"],
        bonus: Annotated[int, "Damage bonus"] = 0
    ) -> str:
        """Calculate and return damage result."""
        count, sides = map(int, damage_dice.lower().split('d'))
        rolls = [random.randint(1, sides) for _ in range(count)]
        total = sum(rolls) + bonus
        return f"Damage roll: {rolls} + {bonus} = {total} damage"

# Configure kernel with plugins
kernel.add_plugin(DicePlugin(), "dice")
kernel.add_plugin(CombatPlugin(), "combat")

# Configure function auto-invocation
settings = kernel.get_prompt_execution_settings_from_service_id(service_id="agent")
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

### 3. Define Game Master Instructions (Experiment with these!)

Try modifying these instructions to create different game styles!

In [ ]:
game_master_instructions = """
You are a Game Master running a fantasy RPG adventure.

Style:
- Descriptive and engaging narration
- Dynamic responses to player actions
- Balance between combat and roleplay

Use available functions:
- dice.roll_dice: For skill checks and random outcomes
- dice.random_event: For generating unexpected situations
- combat.attack_roll: For combat actions
- combat.damage_roll: For determining damage

Gameplay Guidelines:
1. Describe scenes vividly
2. Respond to player choices
3. Use game mechanics appropriately
4. Maintain narrative flow

Format responses as:
📖 Narration: [scene description]
🎲 Mechanics: [any dice rolls or combat]
❓ Options: [available player actions]
"""

# Create Game Master agent
game_master = ChatCompletionAgent(
    service_id="agent",
    kernel=kernel,
    name="GameMaster",
    instructions=game_master_instructions,
    execution_settings=settings
)

### 4. Example Adventure

Try this example, then modify the instructions and plugins to create different adventures!

In [ ]:
# Start an adventure
game_chat = ChatHistory()
print("Starting Adventure...\n")

# Initial scene
await chat_with_agent(
    game_master,
    "I am a wandering warrior entering the forest. What do I encounter?",
    game_chat
)

# Player action
await chat_with_agent(
    game_master,
    "I want to investigate the strange sounds I hear.",
    game_chat
)

# Combat scenario
await chat_with_agent(
    game_master,
    "I draw my sword and prepare to fight!",
    game_chat
)

# Resolution
await chat_with_agent(
    game_master,
    "After the battle, I search the area for treasure.",
    game_chat
)

### 5. Additional exercises

Try these exercises to experiment with game mechanics:

1. Modify the DicePlugin to add:
   - Skill check system
   - Critical success/failure effects
   - Custom dice combinations

2. Enhance the CombatPlugin with:
   - Different weapon types
   - Defense mechanics
   - Status effects

3. Create new plugins for:
   - Character creation
   - Inventory management
   - Magic system

4. Modify the Game Master instructions to create:
   - Different game genres (sci-fi, horror, etc.)
   - Various difficulty levels
   - Unique narrative styles

5. Advanced Challenge:
   - Create a multi-player system
   - Add persistent character progression
   - Implement branching storylines

Remember: Focus on how plugins and instructions work together to create engaging gameplay!